<a href="https://colab.research.google.com/github/dhruvchopra2003/ANPR/blob/main/FinalYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the YOLO Model

### Installing dependancies

In [8]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [9]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.6/107.7 GB disk)


In [10]:
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Using the pre-trained model: meter4/best

In [3]:
from ultralytics import YOLO
model = YOLO("/content/gdrive/MyDrive/YOLO_v8/training_results/Meter4/weights/best.pt")

storing validation results

In [5]:
result_val = model.predict(source="/content/gdrive/MyDrive/YOLO_v8/data/val/images", save_txt=True, save=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 11130228 parameters, 0 gradients, 28.5 GFLOPs
Results saved to runs/detect/predict
15 labels saved to runs/detect/predict/labels


In [6]:
!cp -r /content/runs/detect/predict /content/gdrive/MyDrive/YOLO_v8/output/validationResults

storing the training results

In [7]:
train_results = model.predict(source="/content/gdrive/MyDrive/YOLO_v8/data/train/images", conf=0.2, save_txt=True, save=True)
!cp -r /content/runs/detect/predict /content/gdrive/MyDrive/YOLO_v8/output

Results saved to runs/detect/predict
403 labels saved to runs/detect/predict/labels


# Sorting the Labels in the images and extracting the numbers
(works best on vertical images)

### Required functions

In [1]:
def chk_horizontal(sorted_labels):
  if float(sorted_labels[1][1]) / float(sorted_labels[3][1]) > 0.9:
    return 1

In [2]:
def sort_y(path):
  with open(path,'r') as f:
    lines = f.read()

  lines = lines[:-1]
  lines = lines.split()

  labels = []
  for i in range(0, len(lines), 5):
    labels.append(lines[i:i+5])
  sorted_labels = sorted(labels, key = lambda x:x[2])

  print("Horizontal labels sorted")

  for i in range(len(sorted_labels) - 1):
    try:
      if (float(sorted_labels[i+1][2]) - float(sorted_labels[i][2])) <= 0.025:
        print("removing: ", sorted_labels[i][2])
        sorted_labels.remove(sorted_labels[i])
        i = i+1
    except:
        print("Removed")

  return sorted_labels

In [3]:
def sort_x (path):
  ''' 
    function to sort the values in the saved labels folder and sort them
    according to the top-left point to get the number.
  '''
  with open (path, 'r') as f:
    lines = f.read()

  lines = lines[:-1]
  lines = lines.split()

  labels = []
  for i in range(0, len(lines), 5):
    labels.append(lines[i:i+5])

  sorted_labels = sorted(labels, key = lambda x:x[1])

  print("labels are sorted")
  if chk_horizontal(sorted_labels)==1:
    print("Image is horizontal. Horizontally checking...")
    return sort_y(path)
# Returning the sorted labels in y direction if image found to be horizontal    

  for i in range(len(sorted_labels) - 1):
    try:
      if (float(sorted_labels[i+1][1]) - float(sorted_labels[i][1])) <= 0.010:
        print("removing: ", sorted_labels[i][1])
        sorted_labels.remove(sorted_labels[i])
        i = i+1
    except:
        print("Removed")

  return sorted_labels

In [4]:
def saveSorted (path, labels): 
  
  with open(path, 'w') as f:
    for sublist in labels:  
      
      line = ' '.join(str(item) for item in sublist)
      f.write(line + '\n')

  print(f"file saved at {path}")

In [5]:
def get_number (list):

  no = []
  for i in range(len(list)):
      no.append(list[i][0])

  for i in range(len(no)):
    if no[i] == '11':
      no[i] = '-'
    elif no[i] == '10':
      no[i] = '.'
  
  flag=0
  minus=0
  
  # mainly for horizontal images
  while no[-1] == '-' or no[-1] == '.' or no[0] =='.':
    if no[-1] == '-':
      no = no[::-1]

    # if no[-1] == '.':
    #   flag = 1
    #   no.remove(no[-1])

    if no[0] == '-' and no[1]=='-':
      no.remove(no[0])

    if no[1] == '.':
      no = no[::-1]

  number = float(''.join(map(str, no)))
  
  if flag == 1:
    number = number/10
  if minus == 1:
    number = (-number)
  return number

In [6]:
def saveNo(path, num):
  with open(path, 'a') as f:
    f.write(str(num))
  print("number added")

### Checking the files and saving them

In [11]:
for i in range (390, 405):
  file_name = i
  
  sorted_labels = sort_x(f"/content/gdrive/MyDrive/YOLO_v8/output/validationResults/labels/{file_name}.txt")
  saveSorted(f'/content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/{file_name}Srt.txt', sorted_labels)
  saveNo(f'/content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/{file_name}Srt.txt', get_number(sorted_labels))

  print(f"The number is: {get_number(sorted_labels)}")
  print("_________________________________________________________________")

labels are sorted
Image is horizontal. Horizontally checking...
Horizontal labels sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/390Srt.txt
number added
The number is: 19.1
_________________________________________________________________
labels are sorted
Image is horizontal. Horizontally checking...
Horizontal labels sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/391Srt.txt
number added
The number is: 245.0
_________________________________________________________________
labels are sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/392Srt.txt
number added
The number is: 2.9
_________________________________________________________________
labels are sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/output/validationResults/Sorted_labels/393Srt.txt
number added
The number is: -1.28
_________________________________________________________________
label

In [7]:
import os

directory = "/content/gdrive/MyDrive/YOLO_v8/output/predict/labels"
cnt = 0
for filename in os.scandir(directory):
  if filename.is_file():
    cnt+=1
print(cnt)

403


In [34]:
i = 1
while i < cnt:
    try:
      sorted_labels = sort_x(f"/content/gdrive/MyDrive/YOLO_v8/output/predict/labels/{i}.txt")
      saveSorted(f'/content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/{i}Srt.txt', sorted_labels)
      saveNo(f'/content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/{i}Srt.txt', get_number(sorted_labels))
      print(f"The number is: {get_number(sorted_labels)}")
    except:
      print("the image needs correction")
    print("_________________________________________________________________")
    i+=1

labels are sorted
removing:  0.241325
removing:  0.280757
removing:  0.481073
Removed
Removed
Removed
file saved at /content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/1Srt.txt
number added
The number is: 1111.48
_________________________________________________________________
labels are sorted
removing:  0.129237
removing:  0.278602
removing:  0.591102
Removed
Removed
Removed
file saved at /content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/2Srt.txt
the image needs correction
_________________________________________________________________
labels are sorted
Image is horizontal. Horizontally checking...
Horizontal labels sorted
removing:  0.633333
Removed
file saved at /content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/3Srt.txt
number added
The number is: 0.041
_________________________________________________________________
labels are sorted
Image is horizontal. Horizontally checking...
Horizontal labels sorted
removing:  0.625
removing:  0.630625
removi

In [17]:
import os

directory = "/content/gdrive/MyDrive/YOLO_v8/data/val/labels"
cntV = 0
for filename in os.scandir(directory):
  if filename.is_file():
    cntV+=1
print(cntV)

388


In [19]:
for i in range (390, 405):
  file_name = i
  try:
    sorted_labels = sort_x(f"/content/gdrive/MyDrive/YOLO_v8/data/val/labels/{i}.txt")
    saveSorted(f'/content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/{i}Srt.txt', sorted_labels)
    saveNo(f'/content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/{i}Srt.txt', get_number(sorted_labels))
    print(f"The number is: {get_number(sorted_labels)}")
  except:
    print("the image needs correction")
  print("_________________________________________________________________")
  i+=1

labels are sorted
Image is horizontal. Horizontally checking...
Horizontal labels sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/390Srt.txt
number added
The number is: 16.1
_________________________________________________________________
labels are sorted
removing:  0.076875
removing:  0.085625
Removed
Removed
file saved at /content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/391Srt.txt
the image needs correction
_________________________________________________________________
labels are sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/392Srt.txt
number added
The number is: 2.9
_________________________________________________________________
labels are sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/393Srt.txt
number added
The number is: -1.28
_________________________________________________________________
labels are sorted
file saved at /content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/394Srt.txt
number added


# Storing the results in an excel file

Columns to be added in the excel file

- Image address
- Image preview
- Bounding box image
- Predicted readings
- Actual readings

In [12]:
import os
from PIL import Image

path = "/content/gdrive/MyDrive/YOLO_v8/output/validationResults"
path2 = "/content/gdrive/MyDrive/YOLO_v8/output/predict"

# resizing images and saving into the resized folder
def Img_resize(Img_path, resizeimg_path):
    ##resizing image to insert in excel file
    basewidth = 160
    img = Image.open(Img_path)
    wpercent = (basewidth / float(img.size[0]))
    # hsize = int((float(img.size[1]) * float(wpercent)))
    hsize = 200
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img.save(resizeimg_path)
    # resize path is /content/gdrive/MyDrive/YOLO_v8/output/validationResults/resized_images



for img in os.listdir(path):
  if img.endswith(".jpg"):
    img_path = os.path.join(path, img)
    resize_path = f"/content/gdrive/MyDrive/YOLO_v8/output/validationResults/resized_images/{img}"
    Img_resize(img_path, resize_path)



for img in os.listdir(path2):
  if img.endswith(".jpg"):
    img_path = os.path.join(path2, img)
    resize_path = f"/content/gdrive/MyDrive/YOLO_v8/output/predict/resized_images/{img}"

    Img_resize(img_path, resize_path)


for img in os.listdir("/content/gdrive/MyDrive/YOLO_v8/data/train/images"):
  if img.endswith(".jpg"):
    img_path = os.path.join("/content/gdrive/MyDrive/YOLO_v8/data/train/images", img)
    resize_path = f"/content/gdrive/MyDrive/YOLO_v8/data/resized_images/{img}"

    Img_resize(img_path, resize_path)

In [24]:
for img in os.listdir("/content/gdrive/MyDrive/YOLO_v8/data/val/images"):
  if img.endswith(".jpg"):
    img_path = os.path.join("/content/gdrive/MyDrive/YOLO_v8/data/val/images", img)
    resize_path = f"/content/gdrive/MyDrive/YOLO_v8/data/resized_images/{img}"

    Img_resize(img_path, resize_path)

In [13]:
!pip install xlsxwriter
import xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 9.3 MB/s eta 0:00:00


In [36]:
def Save_in_ExcelFile(drive_path):
  workbook = xlsxwriter.Workbook("Final2.xlsx")
  worksheet = workbook.add_worksheet()
  worksheet.write('A1', "Image_Link")
  worksheet.write('B1', "Image_Preview")
  worksheet.write('C1', "BoundingBox_Image")
  worksheet.write('D1', "Predicted_Readings")
  worksheet.write('E1', "Actual_Readings")

  format = workbook.add_format({'text_wrap':True})
  worksheet.set_column('A:A', 15)
  worksheet.set_column('B:B', 25)
  worksheet.set_column('C:C', 25)
  worksheet.set_column('D:D', 15)
  worksheet.set_column('E:E', 15)



  # setting a counter

  img_address_path = drive_path + '/content/gdrive/MyDrive/YOLO_v8/data/train/images'

  counter = 1
  for i in os.listdir(drive_path + "output/predict/resized_images"):
    if i.endswith('.jpg'):
      counter += 1
  cnt = 0
  while cnt <= counter:   
    cnt +=1
    worksheet.set_row(cnt-1, 404)
    
    worksheet.write("A" + str(cnt), f"{img_address_path}/{cnt}.jpg", format)

    worksheet.insert_image("B" + str(cnt), f'/content/gdrive/MyDrive/YOLO_v8/data/resized_images/{cnt}.jpg')

    worksheet.insert_image("C" + str(cnt), f'/content/gdrive/MyDrive/YOLO_v8/output/predict/resized_images/{cnt}.jpg')

    # Getting predicted and actual readings
    try:
      a = f"/content/gdrive/MyDrive/YOLO_v8/data/Sorted_labels/{cnt}Srt.txt"
      with open (a, 'r') as f:
        line = f.read()

      line = line.split()
      actual_val = line[-1]

      worksheet.write("E" + str(cnt), actual_val)
    except:
      worksheet.write("E"+str(cnt), "File Not Found")


    try:
      x = f"/content/gdrive/MyDrive/YOLO_v8/output/predict/Sorted_labels/{cnt}Srt.txt"

      with open (x, 'r') as f:
        lines = f.read()

      # lines = lines[:-1]
      lines = lines.split()
      predicted_value = lines[-1]

      worksheet.write("D" + str(cnt), predicted_value)
    except:
      worksheet.write("D"+str(cnt), "File Not Found")
  
  workbook.close()

In [37]:
Save_in_ExcelFile("/content/gdrive/MyDrive/YOLO_v8/")

/usr/local/lib/python3.9/dist-packages/xlsxwriter/worksheet.py:1454: UserWarning: Image file '/content/gdrive/MyDrive/YOLO_v8/data/resized_images/405.jpg' not found.
  warn("Image file '%s' not found." % filename)
/usr/local/lib/python3.9/dist-packages/xlsxwriter/worksheet.py:1454: UserWarning: Image file '/content/gdrive/MyDrive/YOLO_v8/output/predict/resized_images/405.jpg' not found.
  warn("Image file '%s' not found." % filename)
/usr/local/lib/python3.9/dist-packages/xlsxwriter/worksheet.py:1454: UserWarning: Image file '/content/gdrive/MyDrive/YOLO_v8/data/resized_images/406.jpg' not found.
  warn("Image file '%s' not found." % filename)
/usr/local/lib/python3.9/dist-packages/xlsxwriter/worksheet.py:1454: UserWarning: Image file '/content/gdrive/MyDrive/YOLO_v8/output/predict/resized_images/406.jpg' not found.
  warn("Image file '%s' not found." % filename)
